<center>

<h1 style="text-align:center"> Higher Order Programming </h1>
</center>

* Higher order functions take other functions as inputs or return other functions as output (or both).
    + All multi-argument functions in OCaml are actually higher-order.
* Also known as *functionals*
    + Hence *functional programming* at its heart is programming with higher-order functions.

## Double and Square

In [ ]:
let double x = 2 * x
let square x = x * x

In [ ]:
double 10

In [ ]:
square 2

## Quad and Fourth

In [ ]:
let quad x = 2 * 2 * x
let fourth x = (x * x) * (x * x)

In [ ]:
quad 10

In [ ]:
fourth 2

## Quad and Fourth

Abstract away the details using `double` and `square`.

In [ ]:
let quad x = double (double x)

In [ ]:
quad 10

In [ ]:
let fourth x = square (square x)

In [ ]:
fourth 2

## Quad and Fourth

Abstract the act of applying twice.

In [ ]:
let twice f x = f (f x)

In [ ]:
let quad x = twice double x

In [ ]:
let quad = twice double

In [ ]:
quad 10

## Quad and Fourth

Abstract the act of applying twice.

In [ ]:
let fourth = twice square

In [ ]:
fourth 2

## The Abstraction Principle in software engineering

* Find similarities and then abstract them out into functions (or other units of code)
  + Also called refactoring in software engineering
* Avoid repeating the same thing in code more than once.
  + Factor out the recurring pattern.
* Advantages of refactoring:
  + Reduces code size
  + Improves readability
  + Decreases the possibility of errors
  + Simplifies code change
* In the previous example, if there was a more efficient implementation of `twice`, both `quad` and `fourth` will benefit from the efficiency.


## Applying a function for an arbitrary number of times

Instead of twice, what if we wanted to apply `n` time over an argument where `n` is supplied as an argument

In [ ]:
let rec apply n f x =
  if n = 0 then x
  else f (apply (n-1) f x)

In [ ]:
let quad = apply 2 double
let _ = quad 10

In [ ]:
quad 10

## Higher Order Programming over Lists

<center>

<h1> Map </h1>
<h1> & </h1>
<h1> Fold </h1>
<h4> (sibling of reduce) </h4>
</center>

<center>

<img src="images/map_reduce_new_yorker.png">
</center>

## MapReduce

<h3> "[Google’s MapReduce] abstraction is inspired by the map and reduce primitives present in Lisp and many other <span style="color:orange"> <i> functional languages. </i> </span>" </h3>

<h4 style="text-align:right"> [Dean and Ghemawat, 2008] </h4>


## Map

`List.map` takes a list `[a1; a2; ...; an]` and a higher-order function `f` and returns `[f a1; f a2; ...; f an]`.

In [1]:
List.map

- : ('a -> 'b) -> 'a list -> 'b list = <fun>


In [2]:
List.map (fun x -> x + 1) [1;2;3]

- : int list = [2; 3; 4]


In [3]:
List.map (fun x -> x ^ "!") ["sweet";"salty"]

- : string list = ["sweet!"; "salty!"]


## Map

In [ ]:
let rec map f l =
  match l with
  | [] -> []
  | x::xs -> f x :: (map f xs)

Is there a problem with this implementation?

* Not tail recursive. 
  + Generally not an issue in practice.
  + Recursion depth bound by the size of the list. 

## rev_map

In [ ]:
let rec rev_map f l acc = 
  match l with
  | [] -> acc
  | x::xs -> rev_map f xs (f x::acc)

In [ ]:
let map f l =
  List.rev (rev_map f l [])

In [ ]:
map (fun x -> x + 1) [1;2;3]

## Fold

* Fold is a function for combining elements. 
* Fold is very powerful => very generic / difficult to understand.
* Let's take a simple example first.

In [4]:
let rec sum l = 
  match l with
  | [] -> 0
  | h :: t -> h + sum t
  
let s = sum [1;2;3]

val sum : int list -> int = <fun>


val s : int = 6


In [5]:
let rec concat l = 
  match l with
  | [] -> ""
  | h :: t -> h ^ concat t
  
let c = concat ["a";"b";"c"]

val concat : string list -> string = <fun>


val c : string = "abc"


Let's try to apply the abstraction principle!

In [7]:
let rec sum' f init l = 
  match l with
  | [] -> init
  | h :: t -> f h (sum' f init t)

let sum = sum' (+) 0

let s = sum [1;2;3]

val sum' : ('a -> 'b -> 'b) -> 'b -> 'a list -> 'b = <fun>


val sum : int list -> int = <fun>


val s : int = 6


In [8]:
let rec concat' f init l = 
  match l with
  | [] -> init
  | h :: t -> f h (concat' f init t)
  
let concat = concat' (^) ""

let c = concat ["a";"b";"c"]

val concat' : ('a -> 'b -> 'b) -> 'b -> 'a list -> 'b = <fun>


val concat : string list -> string = <fun>


val c : string = "abc"


## fold_right

In [9]:
let rec fold_right f l acc = 
  match l with
  | [] -> acc
  | h :: t -> f h (fold_right f t acc)

val fold_right : ('a -> 'b -> 'b) -> 'a list -> 'b -> 'b = <fun>


Another way to look at `fold_right f [a;b;c] acc`:
* `[a;b;c]` is equivalent to `:: a (:: b (:: c []))`
* Replace every occurrence of `::` with `f` and `[]` with `acc`
  + `f a (f b (f c acc))`
* Intuitively, `fold_right` combines the elements of the list from right to left, using the input binary function.

## Abstraction principle in action

In [11]:
let sum l = fold_right (fun x y -> x + y) l 0

let s = sum [1;2;3]

val sum : int list -> int = <fun>


val s : int = 6


In [12]:
let concat l = fold_right (fun x y -> x ^ y) l ""

let c = concat ["a";"b";"c"]

val concat : string list -> string = <fun>


val c : string = "abc"


## Tail recursive fold_right

Let's look at the implementation of `fold_right` again:

```ocaml
let rec fold_right f l acc = 
  match l with
  | [] -> acc
  | h :: t -> f h (fold_right f t acc)
```

Is it tail-recursive?

Let's try to make it tail-recursive:

In [13]:
let rec fold_right_tailrec f l acc = 
  match l with
  | [] -> acc
  | h :: t -> fold_right_tailrec f t (f h acc)

val fold_right_tailrec : ('a -> 'b -> 'b) -> 'a list -> 'b -> 'b = <fun>


Is this equivalent to `fold_right`?

## Tail recursive fold_right

In [14]:
let _ = fold_right_tailrec (+) [1;2;3] 0

- : int = 6


In [15]:
let _ = fold_right_tailrec (^) ["a";"b";"c"] ""

- : string = "cba"


* In the recursive case of `fold_right`, we first combine all the elements of the tail, and then combine the result with the head.
  + The same value of `acc` is passed all the way to the end of the list, which is then combined with the last element.
* In the recursive case of `fold_right_tailrec`, we eagerly combine the head with the accumulator so far, and then we fold in the elements from the tail.

## fold_left

Tail recursive `fold_right` is called `fold_left`:

```ocaml
let rec fold_left f acc l = 
  match l with
  | [] -> acc
  | h :: t -> fold_left f (f h acc) t
```

* Note that by convention, the accumulator is the second argument in `fold_left`.
  + It was the third argument in `fold_right`
+ `fold_left` folds from left to right starting with the initial value of the accumulator.

## fold_left and fold_right in the List module

Let's take a closer look at the types of the two fold functions:

In [16]:
List.fold_right

- : ('a -> 'b -> 'b) -> 'a list -> 'b -> 'b = <fun>


In [17]:
List.fold_left

- : ('a -> 'b -> 'a) -> 'a -> 'b list -> 'a = <fun>


## Behold the power of fold

Any time you need to traverse the list, you can use `fold`.

In [ ]:
let rev l = fold_left (fun acc x -> x :: acc) [] l

In [ ]:
let length l = fold_left (fun acc _ -> acc + 1) 0 l

In [ ]:
let map f l = fold_right (fun x acc -> (f x) :: acc) l []

* `map` is not tail recursive since `fold_right` is not a tail recursive function. 

## Exercise

Implement `exists : ('a -> bool) -> 'a list -> bool` function using `fold_left`. `exists p l` returns `true` if there exists an element `e` in `l` such that `p e` is true. Otherwise, `exists p l` returns `false`.

In [21]:
let exists p l = failwith "not implemented"


val exists : 'a -> 'b -> 'c = <fun>


In [20]:
assert (exists (fun e -> e = 0) [1;3;0] = true)

- : unit = ()


## Exercise

Implement `append : 'a list -> 'a list -> 'a list` using `fold_right`. 

In [24]:
let append l1 l2 = failwith "not implemented"

val append : 'a -> 'b -> 'c = <fun>


In [23]:
assert (append [1;2] [3;4] = [1;2;3;4])

- : unit = ()
